# Market Basket Analysis

### Zumarsiyah Mahsyari
##### Astra Data Scientist Bootcamp

Market basket analysis merupakan salah satu penggunaan teknik asosiasi yang digunakan untuk menemukan kelompok-kelompok barang yang terjadi secara bersamaan dalam suatu transaksi (Witten, Frank, & Hall, 2011). Sebagai contoh, market basket analysis dapat mengungkapkan fakta bahwa konsumen yang membeli teh juga membeli makanan ringan atau fakta lainnya. Fakta-fakta ini dapat menjadi sudut pandang baru bagi pihak supermarket untuk menyusun strategi pengembangan bisnis mereka.

Algoritma yang digunakan pada Practice Case kali ini adalah Apriori. Algoritma ini didasarkan pada frequent itemsets yang diketahui sebelumnya dan fokus pada hubungan antar item dalam keseluruhan transaksi. Adapun data yang digunakan adalah dataset 'Online Retail' yang merupakan data transaksi dari sebuah pusat grosir yang melayani transaksi di beberapa negara. Data ini terdiri dari 8 variabel, yaitu: <br>
1. Invoice No : Nomor nota pada setiap pembelian
2. Stock Code : Merupakan kode stok dari sebuah barang
3. Description : Deskripsi dari barang tersebut
4. Quantitiy : Jumlah barang yang di beli pada sebuah transaksi.
5. Invoice date : tanggal pembelian
6. Unit Price : Harga per unit barang
7. Customer ID : ID Dari pelanggan
8. Country : Negara dari pelangan.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

ModuleNotFoundError: No module named 'mlxtend'

### Load & Read Data

In [3]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

### Preprocessing Data

In [4]:
#Menghilangkan beberapa spasi pada 'Description'
df['Description'] = df['Description'].str.strip()
#Menghapus transaksi yang data 'InvoiceNo'-nya kosong
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
#Mengubah feature 'InvoiceNo' menjadi tipe string
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
#Menghilangkan transaksi yang merupakan transaksi kredit (Di awali dengan hufuf C pada invoice number)
df = df[~df['InvoiceNo'].str.contains('C')]

In [5]:
#Mengambil data transaksi yang berasal dari negara Prancis berdasarkan 'InvoiceNo' dan menyiapkan struktur data biner
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [6]:
#Membuat data biner
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

Data biner didapatkan dengan melakukan encoding seperti di atas. Apabila data berisi 0, maka pada transaksi dengan 'InvoiceNo' tersebut tidak terjadi pembelian barang yang bersangkutan. Begitu sebaliknya, apabila data berisi 1, maka pada transaksi dengan 'InvoiceNo' tersebut terjadi pembelian barang yang bersangkutan. 

Minimal support ditentukan oleh peneliti. Semakin besar minimal support, semakin sedikit akan kita dapatkan frequent itemsets. Kali ini kita menggunakan minimal support sebesar 0,07.

In [7]:
#Mendapatkan barang yang sering dibeli
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

### Association Rules

In [8]:
#Mendapatkan rules barang
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

antecedents                   consequents  \
0   (ALARM CLOCK BAKELIKE PINK)  (ALARM CLOCK BAKELIKE GREEN)   
1  (ALARM CLOCK BAKELIKE GREEN)   (ALARM CLOCK BAKELIKE PINK)   
2    (ALARM CLOCK BAKELIKE RED)  (ALARM CLOCK BAKELIKE GREEN)   
3  (ALARM CLOCK BAKELIKE GREEN)    (ALARM CLOCK BAKELIKE RED)   
4   (ALARM CLOCK BAKELIKE PINK)    (ALARM CLOCK BAKELIKE RED)   

   antecedent support  consequent support   support  confidence      lift  \
0            0.102041            0.096939  0.073980    0.725000  7.478947   
1            0.096939            0.102041  0.073980    0.763158  7.478947   
2            0.094388            0.096939  0.079082    0.837838  8.642959   
3            0.096939            0.094388  0.079082    0.815789  8.642959   
4            0.102041            0.094388  0.073980    0.725000  7.681081   

   leverage  conviction  
0  0.064088    3.283859  
1  0.064088    3.791383  
2  0.069932    5.568878  
3  0.069932    4.916181  
4  0.064348    3.293135

Kali ini kita tentukan minimal lift sebesar 1. Jika nilai lift >= 1, maka association rules tersebut memiliki hubungan yang signifikan.

In [2]:
#Association rules dengan nilai lift >= 6 dan nilai confidence >= 0.8
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

NameError: name 'rules' is not defined

Menghitung jumlah barang pada transaksi

In [12]:
#Menghitung jumlah ALARM CLOCK BAKELIKE GREEN pada transaksi
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

340.0

In [11]:
##Menghitung jumlah ALARM CLOCK BAKELIKE RED pada transaksi
basket['ALARM CLOCK BAKELIKE RED'].sum()

316.0

Sekarang mari kita coba dapatkan association rules untuk transaksi yang terjadi di jerman dengan minimal support sebesar 0,05; nilai lift >= 4; dan nilai confidence >= 0,5.

In [13]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

antecedents                         consequents  \
0   (PLASTERS IN TIN CIRCUS PARADE)  (PLASTERS IN TIN WOODLAND ANIMALS)   
6        (PLASTERS IN TIN SPACEBOY)  (PLASTERS IN TIN WOODLAND ANIMALS)   
10    (RED RETROSPOT CHARLOTTE BAG)            (WOODLAND CHARLOTTE BAG)   

    antecedent support  consequent support   support  confidence      lift  \
0             0.115974            0.137856  0.067834    0.584906  4.242887   
6             0.107221            0.137856  0.061269    0.571429  4.145125   
10            0.070022            0.126915  0.059081    0.843750  6.648168   

    leverage  conviction  
0   0.051846    2.076984  
6   0.046488    2.011670  
10  0.050194    5.587746